<a href="https://colab.research.google.com/github/nehasupe/AppliedMachinelearning/blob/master/assignment4_F19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CSCI P-556: Applied Machine Learning

Fall 2019

Assignment 4

Due: 11:59PM, December 6, 2019

Read dataset

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
train = pd.read_csv('a4-train.csv', index_col=0)
test = pd.read_csv('a4-test.csv', index_col=0)

y_train = train['labels']
x_train = train.drop(['labels'], axis=1)
 

y_test = test['labels']
x_test = test.drop(['labels'], axis=1)


Task 1: Perform exploratory data analysis (15 points)

Reference: https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15

On performing exploratory data analysis following is the inferrence:
Both the train set and test set are balanced. Since the labels in the train set are balanced, we can use accuracy as a metric to evaluate our models. All the 500 features are numerical features of int64 type.There are no missing values. The heatmap shows a few of the features have a high correlation.

In [0]:
# Train set
print("Exploratory Data Analysis:")
print("Train set:")
print("Train Shape:", train.shape)
print("Number of samples:", train.shape[0])
print("Number of features:", train.shape[1])
print("Train head, 5 first samples")
print(train.head())
print("Train information:")
print(train.info())

print("Train description:")
print(train.describe())

print("Target variable:")
print(" target labels", train.labels.unique())
print("Each label count:")
print(train.labels.value_counts())



In [0]:
print("Graphical analysis of data:")
# Reference: https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15
sns.pairplot(x_train, hue = 'labels', size = 10)
plt.show()
print("Heatmap for missing values:")
sns.heatmap(train.isnull(),cbar=False,yticklabels=False,cmap = 'viridis')
plt.figure(figsize=(60,40))
sns.heatmap(train.corr(),cmap='Blues',annot=False)

print("Correlation Heatmap:")
k = 501 #number of variables for heatmap
cols = train.corr().nlargest(k, 'labels')['labels'].index
cm = train[cols].corr()
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, cmap = 'viridis')



In [0]:

# Test set

print("Test set")

print("Test shape", test.shape)
print("Test head")
print(test.head())
print("Test Information")
print(test.info())
print("Test Description")
print(test.describe())
print("Test target variable labels")
print(test.labels.unique())
print("Test target variable label counts")
print(test.labels.value_counts())


In [0]:
# Reference: https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15
print("Graphical analysis of data:")
print("Heatmap for missing values in test set")
sns.heatmap(test.isnull(),cbar=False,yticklabels=False,cmap = 'viridis')
plt.figure(figsize=(60,40))
sns.heatmap(test.corr(),cmap='Blues',annot=False)

print("Heatmap showing correlation amongst the features:")
k = 501 #number of variables for heatmap
cols = test.corr().nlargest(k, 'labels')['labels'].index
cm = test[cols].corr()
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, cmap = 'viridis')


We can observe that there are no missing values.
The train as well as test set is balanced # what to do of negative correlation values

Task 2: Use scikit-learn's logistic regression to establish a baseline model. We are not expecting this model to perform well, we just want to know what is the highest accuracy that we can achieve without doing any feature engineering and/or parameter tuning. (5 points)

On performing Logistic Regression on the given data without any feature engineering gives accuracy of 59% on the test set.

In [0]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_predict = logreg.predict(x_test)

train_score = logreg.score(x_train, y_train)
test_score = logreg.score(x_test, y_test)
print("Train set score:", train_score)
print("Test set score", test_score)
# Train set score: 0.745
# Test set score 0.59

Task 3: Feature engineering. On this task you'll write code to modify your features such that the model can achieve a higher accuracy. You are free to modify the data as you want, but a rationale has to be provided for each modification. Additionally, if you are using anything that has not been convered in class, write a brief description of how the function works. Note that copy/pasting a function's docstring is not what we are asking and we will take off points if this is what you do. (20 points)


I have tried different feature selection methods and then selected ExtraTreeClassfier feature importance method for which my models gave the highest accuracy. I am showing all the methods that I have tried for feature engineering and have commented them. They do work in improving the results but did not result in the highest accuracy

Feature Engineering makes use of domain information of features to train models to solve problems. This could mean creating new features or selecting from existing features. Since all the features are numerical features which have continuous values, and the number of features is high as compared to the given number of samples, to train models it made sense to reduce features than adding new features. Without a context of the features and more information on the numerical values, its not easy to create features. The information we can infer looking at the dataset is that this is a classification problem since our target variable is always 0 or 1.

Reference: https://scikit-learn.org/stable/modules/feature_selection.html
The following methods have been used to reduce the number of features simply beacuse not all features contribute equally in helping classify a sample and features which do not contribute enough can be eliminated.

1. ExtraTreesClassifier
2. Dropping highly correlated features 
3. Lasso Regularization
4. Principal Component Analysis
5. SelectKBest
6. Recursive Feature Elimination

**Using feature importance Extra Trees Classifier**

Ref: https://www.geeksforgeeks.org/ml-extra-tree-classifier-for-feature-selection/

Extra Trees Classifier is based on decision trees but has low variance. Here, decision trees are created for each training sample. In these decision trees, the data is split at each node by considering random k features. They aim to find the best features to split the data by calculaing Gini Index.

 Based on the values of Gini Index values used for splitting, a feature importance value is calculated which is normalized reduction in Gini index. This is ordered in non-increasing order and top K features are selected. 

In [0]:
# Reference: https://scikit-learn.org/stable/modules/feature_selection.html
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# Build a forest and compute the feature importances
tree = ExtraTreesClassifier(n_estimators=100,
                              random_state=10)

tree.fit(x_train, y_train)
model = SelectFromModel(tree, prefit=True)
x_train = pd.DataFrame(model.transform(x_train))
x_test = pd.DataFrame(model.transform(x_test))

print("Train shape:", x_train.shape)
print("Test shape",x_test.shape)

Train shape: (2000, 136)
Test shape (600, 136)


**Dropping Highly Correlated Features**

Features which have a correlation above a threshold of 0.95 in the train set are dropped from the train set and test set. One feature of every correlated set of features is kept in the dataset. Features which are highly correlated will not contribute anything new to the target variable of the samples and hence can be dropped.

In [0]:
'''
# https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
# Create correlation matrix
corr_matrix = x_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x_train = x_train.drop(x_train[to_drop], axis=1)
x_test = x_test.drop(x_test[to_drop], axis=1)
print("train shape", x_train.shape)
print("test shape", x_test.shape)
print("Coulmns dropped:", to_drop)


train shape (2000, 278)
test shape (600, 278)
Coulmns dropped: []


**Lasso Regularization**

Reference: https://www.machinelearningplus.com/machine-learning/feature-selection/

Lasso regression works in a way that it imposes a cost to having high coefficient values. It attempts to shrink these large coefficients in Logistic Regression. The coefficients of unwanted/ dummy features reduce to zero altogther, removing these features. 


In [0]:
'''
# Reference: https://scikit-learn.org/stable/modules/feature_selection.html
# Regulariztion

from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

lsvc = LogisticRegression(C=0.01, penalty="l1", dual=False).fit(x_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
x_train = model.transform(x_train)
x_test = model.transform(x_test)
print("Train shape",x_train.shape)
print("Test shape", x_test.shape)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(2000, 345)
(600, 345)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
'''
#### https://towardsdatascience.com/dimension-reduction-techniques-with-python-f36ca7009e5c
# https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
from sklearn.decomposition import PCA

pca = PCA(.95)

pca.fit(x_train)
x_train =pd.DataFrame(pca.transform(x_train))
x_test = pd.DataFrame(pca.transform(x_test))
print(x_train.shape)
print(x_test.shape)

(2000, 278)
(600, 278)


**SelectKBest**


This method takes a score function and number of features to keep as a parameter. Here I have used chi2 as the score function. This function is indicative that if the feature is not dependent on the target variable then it does not contribute in classification of samples and hence will have less score. The model at the end retains K features with the highest scores.

In [0]:
'''
# Ref:  https://scikit-learn.org/stable/modules/feature_selection.html
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# 1. Univariate Selection
# Statistical tests can be 
# used to select those features that have the strongest relationship with the output variable.

model = SelectKBest(chi2, k = 100)
model.fit(x_train, y_train)
x_train =pd.DataFrame(model.transform(x_train))
x_test = pd.DataFrame(model.transform(x_test))
print(x_train.shape)
print(x_test.shape)


(2000, 100)
(600, 100)


**Recursive Feature Elimination**

Ref: https://www.scikit-yb.org/en/latest/api/model_selection/rfecv.html#:~:targetText=Recursive%20feature%20elimination%20(RFE)%20is,number%20of%20features%20is%20reached.&targetText=RFE%20requires%20a%20specified%20number,how%20many%20features%20are%20valid.

RFE has been fit to a Support Vector Classifier model. The object of this model is passed as a parameter to the RFE function. Number of features to keep can also be passed as a parameter otherwise default None considers half of the total number of features to keep. RFE ranks the features by the coefficients of the model. They are recursively eliminated based on the rank till the number of required features is reached.

In [0]:
'''
# Recursive Feature elimination
# Takes forever to run 
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

svc = SVC(kernel = 'linear', C=1)
rfe = RFE(estimator=svc, step=1)
rfe.fit(x_train, y_train)
x_train =pd.DataFrame(rfe.transform(x_train))
x_test = pd.DataFrame(rfe.transform(x_test))
print(x_train.shape)
print(x_test.shape)
'''

Task 4: Model building & evaluation. Train at least 4 models:


1.   Neural network (any type of NN is fine)
2.   Decision tree (can be a plain decision tree, random forest, gradient boosted trees, etc.)
3.   Support vector machine
4.   Your choice of Naive Bayes or K-nearest neighbors

For model 4, briefly (no more than 2 paragraphs) describe how the model works. 

Part of your grade will depend on how your best model performs against the best model of all the classmates, as determined by the accuracy achieved using the test set. You are encouraged to share your best model's accuracy on Piazza, but not which feature engineering or model tuning steps you took to achieve it. (60 points)

In [0]:
# Neural network (any type of NN is fine)
# Decision tree (can be a plain decision tree, random forest, gradient boosted trees, etc.)
# Support vector machine
# Your choice of Naive Bayes or K-nearest neighbors

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
print(x_train.shape)
print(x_test.shape)

(2000, 500)
(600, 500)


In [0]:
# Model 1
# Neural Network
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
nn = MLPClassifier(solver = 'adam', alpha=0.1, activation= 'relu', hidden_layer_sizes=(512, 512, 512, 512), random_state=60)
nn.fit(x_train, y_train)
y_pred = nn.predict(x_test)
nn_accuracy = metrics.accuracy_score(y_test, y_pred)
print("nn accuracy:", nn_accuracy)
print("Confusion Matrix", metrics.confusion_matrix(y_test, y_pred))

nn accuracy: 0.73
Confusion Matrix [[262  38]
 [124 176]]


In [0]:
# Model 2 - 
# Decision Tree (Gradient Boosted Trees)
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
tree = GradientBoostingClassifier(n_estimators= 250, min_samples_split= 2, validation_fraction= 0.4, random_state= 6, max_depth= 8, loss= 'deviance', learning_rate= 0.1, min_impurity_decrease = 0.01, max_features=75)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)
tree_accuracy = metrics.accuracy_score(y_test, y_pred)
print("Test accuracy:", tree_accuracy)
print("Confusion Matrix", metrics.confusion_matrix(y_test, y_pred))



Test accuracy: 0.8766666666666667
Confusion Matrix [[261  39]
 [ 35 265]]


In [0]:
# Model 3
# SVM
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
svclassifier = SVC(kernel = 'rbf', C= 1, gamma = 1, shrinking=True) # gamma = 0.1, 1, 10, 100 
svclassifier.fit(x_train, y_train)
y_pred = svclassifier.predict(x_test)
svm_accuracy = metrics.accuracy_score(y_test, y_pred)
print("svm accuracy:", svm_accuracy)
print("Confusion Matrix", metrics.confusion_matrix(y_test, y_pred))


svm accuracy: 0.5
[[  0 300]
 [  0 300]]


In [0]:
# Model 4
# K nearest neighbors
# Ref: https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py
# 
knn = KNeighborsClassifier(n_neighbors = 43, weights = 'distance', algorithm='auto')
# try what is this distance thing
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
knn_accuracy = metrics.accuracy_score(y_test, y_pred)
print("Knn accuracy:", knn_accuracy)
print("Confusion Matrix", metrics.confusion_matrix(y_test, y_pred))


Knn accuracy: 0.8133333333333334
Confusion Matrix [[252  48]
 [ 64 236]]


Best model is Gradient boosted trees with accuracy of 87.66% and using ExtraTreesClassifier to select important features

Model 4:

K nearest neighbors:
This model is called a lazy learner algorithm because at train time it doesn't learn anything. At the time of prediction, for a test sample, it will calculate the distance between this test sample and every sample of the train set. The distances can be euclidean, manhattan etc. It will then sort these distances in increasing order and select the top k distances. These K distances are the neighbors of this test sample and indicate that how similar they are to the test sample. The test sample is classified to the class to which maximum neighbors belong to. 
